# Assignment 3: Adversarial Defense Strategies

In this assignment, you will implement and evaluate practical defense mechanisms against adversarial attacks. You'll learn how to make neural networks more robust by implementing simple yet effective defensive techniques.

**Instructions:**
1. Complete all 3 exercises in this notebook
2. Run all cells and ensure outputs are visible
3. Submit the completed notebook


## Setup

Run the code below to set up the notebook

In [ ]:
# Setup and imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm
import copy

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Enhanced plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print("🎯 Ready to train a fresh model and explore adversarial attacks...")


Run the code below to build and train the machine learning model...

In [ ]:
# Build and Train Target Model for Adversarial Attacks

print("🏗️ Training a fresh CNN model for adversarial attack assignments...")
print("(This ensures we have a well-trained target for meaningful attack demonstrations)")

# Define CNN architecture for MNIST classification
class MNISTNet(nn.Module):
    """Convolutional Neural Network for MNIST digit classification."""
    
    def __init__(self):
        super(MNISTNet, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        
        # Pooling and dropout
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        # First conv block
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout1(x)
        
        # Flatten for fully connected layers
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        
        return x

# Data loading and preprocessing
print("📊 Loading MNIST dataset...")
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load datasets
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform_test)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
test_loader_single = DataLoader(test_dataset, batch_size=1, shuffle=False)

print(f"✅ Dataset loaded: {len(train_dataset)} training, {len(test_dataset)} test samples")

# Initialize model, loss, and optimizer
model = MNISTNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Count parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"🤖 Model parameters: {total_params:,}")

# Quick training function
def train_model(model, train_loader, criterion, optimizer, epochs=3):
    """Train the model for a few epochs to get good performance."""
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        print(f"\n📚 Epoch {epoch+1}/{epochs}")
        
        with tqdm(train_loader, desc=f"Training") as pbar:
            for batch_idx, (data, target) in enumerate(pbar):
                data, target = data.to(device), target.to(device)
                
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                
                # Statistics
                running_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
                
                # Update progress bar
                if batch_idx % 50 == 0:
                    accuracy = 100. * correct / total
                    avg_loss = running_loss / (batch_idx + 1)
                    pbar.set_postfix({
                        'Loss': f'{avg_loss:.4f}', 
                        'Acc': f'{accuracy:.2f}%'
                    })
        
        # Final epoch stats
        epoch_accuracy = 100. * correct / total
        epoch_loss = running_loss / len(train_loader)
        print(f"   📈 Epoch {epoch+1} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

# Test model performance
def evaluate_model(model, test_loader):
    """Evaluate model on test set."""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    accuracy = 100. * correct / total
    return accuracy

# Train the model
print("🚀 Starting training...")
train_model(model, train_loader, criterion, optimizer, epochs=3)

# Evaluate performance
print("\n📊 Evaluating model performance...")
test_accuracy = evaluate_model(model, test_loader)
print(f"🎯 Test Accuracy: {test_accuracy:.2f}%")

if test_accuracy > 95:
    print("✅ Excellent! Model is well-trained and ready for adversarial attacks!")
elif test_accuracy > 90:
    print("✅ Good! Model performance is sufficient for attack demonstrations!")
else:
    print("⚠️  Model accuracy is lower than expected, but will work for demonstrations.")

# Save the trained model for future use
torch.save(model.state_dict(), 'assignment3_trained_model.pth')
print("💾 Model saved as 'assignment3_trained_model.pth'")

print("\n🎯 Model training complete and ready for adversarial attack assignments!")


## Exercise 1: Input Preprocessing Defense

Implement and evaluate input preprocessing techniques that can defend against adversarial attacks. Input preprocessing is one of the simplest defense strategies. The idea is to "clean" the input before feeding it to the model, removing adversarial perturbations while preserving the original signal. Add random Gaussian noise to inputs to disrupt adversarial perturbations.

In [ ]:
def gaussian_noise_defense(image, noise_std=0.1):
    """
    Apply Gaussian noise defense to input image.
    
    Args:
        image: Input image tensor
        noise_std: Standard deviation of Gaussian noise
    
    Returns:
        defended_image: Image with added Gaussian noise
    """
    # TODO: Implement Gaussian noise defense
    # Hint: Use torch.randn_like() to generate noise with same shape as image
    # Add noise to image and clamp to valid range [-1, 1]
    
    # Your implementation here:
    # TODO: Implement your solution here
    
    # TODO: Replace solution above with:
    # pass

## Testing Exercise 1

Run the code below to test your implementation. Note that the defenses will (with high probability) **not** be successful. This is expected, we will discuss better defenses below.

In [ ]:
# Implement stronger attacks for more meaningful defense testing
def fgsm_attack(model, image, label, epsilon=0.3):
    """Fast Gradient Sign Method attack."""
    image.requires_grad_(True)
    output = model(image)
    loss = F.cross_entropy(output, label)
    model.zero_grad()
    loss.backward()
    gradient_sign = image.grad.data.sign()
    adversarial_image = image + epsilon * gradient_sign
    return torch.clamp(adversarial_image, -1, 1).detach()

def pgd_attack(model, image, label, epsilon=0.3, alpha=0.075, iterations=10):
    """
    Projected Gradient Descent attack - much stronger than FGSM.
    
    Args:
        model: Target model
        image: Clean input image
        label: True label
        epsilon: Maximum perturbation budget
        alpha: Step size for each iteration
        iterations: Number of attack iterations
    
    Returns:
        adversarial_image: Adversarially perturbed image
    """
    # Start with a random perturbation
    delta = torch.zeros_like(image).uniform_(-epsilon, epsilon)
    delta = torch.clamp(image + delta, -1, 1) - image
    
    for i in range(iterations):
        delta.requires_grad_(True)
        
        # Forward pass
        output = model(image + delta)
        loss = F.cross_entropy(output, label)
        
        # Backward pass
        model.zero_grad()
        loss.backward()
        
        # Update perturbation
        grad_sign = delta.grad.data.sign()
        delta = delta + alpha * grad_sign
        
        # Project back to epsilon ball
        delta = torch.clamp(delta, -epsilon, epsilon)
        delta = torch.clamp(image + delta, -1, 1) - image
        
        delta = delta.detach()
    
    return (image + delta).detach()

def find_vulnerable_samples(model, test_loader, num_samples=10):
    """Find samples that are not too confident for better attack demonstrations."""
    model.eval()
    vulnerable_samples = []
    
    with torch.no_grad():
        for data, target in test_loader:
            if len(vulnerable_samples) >= num_samples:
                break
                
            data, target = data.to(device), target.to(device)
            output = model(data)
            probs = F.softmax(output, dim=1)
            confidence = probs.max(dim=1)[0]
            
            # Look for samples with 70-95% confidence (not too easy, not too hard)
            for i in range(data.size(0)):
                if 0.7 <= confidence[i] <= 0.95:
                    vulnerable_samples.append((data[i:i+1], target[i:i+1], confidence[i].item()))
                    if len(vulnerable_samples) >= num_samples:
                        break
    
    return vulnerable_samples

# Test the defense functions with stronger attacks
print("🛡️ TESTING INPUT PREPROCESSING DEFENSES")
print("=" * 50)

print("🔍 Finding vulnerable samples for meaningful defense testing...")
vulnerable_samples = find_vulnerable_samples(model, test_loader_single, num_samples=5)

if not vulnerable_samples:
    print("⚠️  No vulnerable samples found, using first available sample...")
    sample_iter = iter(test_loader_single)
    test_image, true_label = next(sample_iter)
    test_image, true_label = test_image.to(device), true_label.to(device)
    vulnerable_samples = [(test_image, true_label, 1.0)]

print(f"✅ Found {len(vulnerable_samples)} samples for testing")

# Test attacks and defenses on multiple samples
total_results = {
    'fgsm_success': 0,
    'pgd_success': 0,
    'gaussian_defense_success': 0,
    'median_defense_success': 0
}

print("\n📊 COMPREHENSIVE ATTACK AND DEFENSE TESTING")
print("=" * 60)

for idx, (test_image, true_label, original_conf) in enumerate(vulnerable_samples):
    print(f"\n🎯 Sample {idx+1}: True label={true_label.item()}, Original confidence={original_conf:.3f}")
    print("-" * 40)
    
    model.eval()
    
    # Test FGSM attack
    fgsm_adv = fgsm_attack(model, test_image.clone(), true_label, epsilon=0.3)
    with torch.no_grad():
        fgsm_output = model(fgsm_adv)
        fgsm_pred = torch.argmax(fgsm_output, dim=1)
        fgsm_conf = F.softmax(fgsm_output, dim=1).max()
    
    fgsm_success = fgsm_pred.item() != true_label.item()
    total_results['fgsm_success'] += fgsm_success
    print(f"FGSM Attack (ε=0.3): Pred={fgsm_pred.item()}, Conf={fgsm_conf:.3f}, Success={fgsm_success}")
    
    # Test PGD attack (stronger)
    pgd_adv = pgd_attack(model, test_image.clone(), true_label, epsilon=0.3, alpha=0.075, iterations=10)
    with torch.no_grad():
        pgd_output = model(pgd_adv)
        pgd_pred = torch.argmax(pgd_output, dim=1)
        pgd_conf = F.softmax(pgd_output, dim=1).max()
    
    pgd_success = pgd_pred.item() != true_label.item()
    total_results['pgd_success'] += pgd_success
    print(f"PGD Attack (ε=0.3): Pred={pgd_pred.item()}, Conf={pgd_conf:.3f}, Success={pgd_success}")
    
    # Use the stronger attack (PGD) for defense testing
    attack_image = pgd_adv if pgd_success else fgsm_adv
    attack_name = "PGD" if pgd_success else "FGSM"
    
    if pgd_success or fgsm_success:
        print(f"\n🛡️ Testing defenses against {attack_name} attack:")
        
        # Test Gaussian noise defense
        defended_gaussian = gaussian_noise_defense(attack_image.clone(), noise_std=0.1)
        with torch.no_grad():
            gaussian_output = model(defended_gaussian)
            gaussian_pred = torch.argmax(gaussian_output, dim=1)
            gaussian_conf = F.softmax(gaussian_output, dim=1).max()
        
        gaussian_success = gaussian_pred.item() == true_label.item()
        total_results['gaussian_defense_success'] += gaussian_success
        print(f"  Gaussian noise (σ=0.1): Pred={gaussian_pred.item()}, Conf={gaussian_conf:.3f}, Defense Success={gaussian_success}")
    else:
        print("  ⚠️  No successful attack to test defenses against")

# Summary statistics
print(f"\n📈 OVERALL RESULTS (across {len(vulnerable_samples)} samples):")
print("=" * 50)
print(f"FGSM Attack Success Rate: {total_results['fgsm_success']}/{len(vulnerable_samples)} ({100*total_results['fgsm_success']/len(vulnerable_samples):.1f}%)")
print(f"PGD Attack Success Rate: {total_results['pgd_success']}/{len(vulnerable_samples)} ({100*total_results['pgd_success']/len(vulnerable_samples):.1f}%)")

successful_attacks = max(total_results['fgsm_success'], total_results['pgd_success'])
if successful_attacks > 0:
    print(f"Gaussian Noise Defense Success: {total_results['gaussian_defense_success']}/{successful_attacks} ({100*total_results['gaussian_defense_success']/successful_attacks:.1f}%)")
    print(f"Median Filter Defense Success: {total_results['median_defense_success']}/{successful_attacks} ({100*total_results['median_defense_success']/successful_attacks:.1f}%)")

print("\n✅ Comprehensive attack and defense testing complete!")
print("💡 PGD attacks are typically much stronger than FGSM - use PGD results for defense evaluation")

## Exercise 2: Adversarial Training Defense 

Implement adversarial training to create a robust model. Adversarial training is one of the most effective defense strategies. The idea is to train the model on both clean and adversarial examples, making it naturally robust to attacks. Create a training loop that includes adversarial examples during training.


In [ ]:
def adversarial_training_step(model, data, target, optimizer, epsilon=0.1):
    """
    Perform one step of adversarial training.
    
    Args:
        model: The neural network to train
        data: Clean input batch
        target: True labels
        optimizer: Optimizer for the model
        epsilon: Perturbation budget for adversarial examples
    
    Returns:
        loss: Combined loss on clean and adversarial examples
    """
    model.train()
    
    # TODO: Implement adversarial training step
    # 1. Generate adversarial examples using FGSM
    # 2. Compute loss on both clean and adversarial examples
    # 3. Combine the losses (e.g., 50% clean + 50% adversarial)
    
    # Your implementation here:
    
    # TODO: Implement your solution here
    
    # TODO: Replace solution above with:
    # pass

## Testing Exercise 2

Run the code below to test your implementation. 

In [ ]:
def train_robust_model(model, train_loader, optimizer, epochs=2, epsilon=0.1):
    """Train a model using adversarial training."""
    print(f"🛡️ Training robust model with adversarial training (ε={epsilon})...")
    
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct_clean = 0
        correct_adv = 0
        total = 0
        
        print(f"\n📚 Robust Training Epoch {epoch+1}/{epochs}")
        
        with tqdm(train_loader, desc="Robust Training") as pbar:
            for batch_idx, (data, target) in enumerate(pbar):
                data, target = data.to(device), target.to(device)
                
                # Adversarial training step
                loss = adversarial_training_step(model, data, target, optimizer, epsilon)
                loss.backward()
                optimizer.step()
                
                # Statistics (evaluate on clean and adversarial examples)
                with torch.no_grad():
                    # Clean accuracy
                    output_clean = model(data)
                    pred_clean = torch.argmax(output_clean, dim=1)
                    correct_clean += (pred_clean == target).sum().item()
                
                # Adversarial accuracy (create fresh adversarial examples)
                # Note: This needs to be outside torch.no_grad() to compute gradients
                data_test_adv = data.clone().detach().requires_grad_(True)
                output_test = model(data_test_adv)
                loss_test = F.cross_entropy(output_test, target)
                model.zero_grad()
                loss_test.backward()
                gradient_sign = data_test_adv.grad.data.sign()
                data_test_adv = data + epsilon * gradient_sign
                data_test_adv = torch.clamp(data_test_adv, -1, 1).detach()
                
                with torch.no_grad():
                    output_adv = model(data_test_adv)
                    pred_adv = torch.argmax(output_adv, dim=1)
                    correct_adv += (pred_adv == target).sum().item()
                
                running_loss += loss.item()
                total += target.size(0)
                
                # Update progress bar
                if batch_idx % 50 == 0:
                    clean_acc = 100. * correct_clean / total
                    adv_acc = 100. * correct_adv / total
                    avg_loss = running_loss / (batch_idx + 1)
                    pbar.set_postfix({
                        'Loss': f'{avg_loss:.4f}',
                        'Clean Acc': f'{clean_acc:.1f}%',
                        'Adv Acc': f'{adv_acc:.1f}%'
                    })
        
        # Final epoch stats
        clean_accuracy = 100. * correct_clean / total
        adv_accuracy = 100. * correct_adv / total
        epoch_loss = running_loss / len(train_loader)
        print(f"   📈 Epoch {epoch+1} - Loss: {epoch_loss:.4f}")
        print(f"   📊 Clean Accuracy: {clean_accuracy:.2f}%")
        print(f"   🛡️ Adversarial Accuracy: {adv_accuracy:.2f}%")

# Create a robust model
print("🏗️ Creating and training a robust model...")
robust_model = MNISTNet().to(device)
robust_optimizer = optim.Adam(robust_model.parameters(), lr=0.001)

# Train with adversarial training
train_robust_model(robust_model, train_loader, robust_optimizer, epochs=2, epsilon=0.1)

# Save the robust model
torch.save(robust_model.state_dict(), 'robust_model.pth')
print("\n💾 Robust model saved as 'robust_model.pth'")
print("✅ Adversarial training complete!")

## Exercise 3: Defense Evaluation and Comparison

Compare the effectiveness of different defense strategies. Now that we have implemented multiple defense approaches, let's systematically evaluate and compare their effectiveness against adversarial attacks.


In [ ]:
def evaluate_defense_robustness(model, test_loader, defense_function, defense_params, num_samples=200):
    """
    Evaluate defense effectiveness against both FGSM and PGD attacks.
    
    Args:
        model: The model to test
        test_loader: Test data loader
        defense_function: Defense function to apply
        defense_params: Parameters for the defense function
        num_samples: Number of samples to test
    
    Returns:
        results: Dictionary with clean and adversarial accuracies for both attacks
    """
    model.eval()
    
    clean_correct = 0
    total = 0
    
    epsilons = [0.1, 0.2, 0.3]  # Different attack strengths
    results = {
        eps: {
            'fgsm_no_defense': 0, 'fgsm_with_defense': 0,
            'pgd_no_defense': 0, 'pgd_with_defense': 0
        } 
        for eps in epsilons
    }
    
    print(f"Testing defense with parameters: {defense_params}")
    
    for i, (data, target) in enumerate(test_loader):
        if total >= num_samples:
            break
            
        data, target = data.to(device), target.to(device)
        
        # Test clean accuracy
        with torch.no_grad():
            clean_output = model(data)
            clean_pred = torch.argmax(clean_output, dim=1)
            clean_correct += (clean_pred == target).sum().item()
        
        # Test against different attack strengths
        for epsilon in epsilons:
            # Test FGSM attack
            fgsm_adv_data = fgsm_attack(model, data.clone(), target, epsilon)
            
            with torch.no_grad():
                # FGSM without defense
                fgsm_output = model(fgsm_adv_data)
                fgsm_pred = torch.argmax(fgsm_output, dim=1)
                correct_fgsm_no_def = (fgsm_pred == target).sum().item()
                results[epsilon]['fgsm_no_defense'] += correct_fgsm_no_def
                
                # FGSM with defense
                if defense_function is not None:
                    fgsm_defended_data = defense_function(fgsm_adv_data.clone(), **defense_params)
                    fgsm_defended_output = model(fgsm_defended_data)
                    fgsm_defended_pred = torch.argmax(fgsm_defended_output, dim=1)
                    correct_fgsm_with_def = (fgsm_defended_pred == target).sum().item()
                    results[epsilon]['fgsm_with_defense'] += correct_fgsm_with_def
                else:
                    results[epsilon]['fgsm_with_defense'] += correct_fgsm_no_def
            
            # Test PGD attack (stronger)
            pgd_adv_data = pgd_attack(model, data.clone(), target, epsilon, alpha=epsilon/4, iterations=10)
            
            with torch.no_grad():
                # PGD without defense
                pgd_output = model(pgd_adv_data)
                pgd_pred = torch.argmax(pgd_output, dim=1)
                correct_pgd_no_def = (pgd_pred == target).sum().item()
                results[epsilon]['pgd_no_defense'] += correct_pgd_no_def
                
                # PGD with defense
                if defense_function is not None:
                    pgd_defended_data = defense_function(pgd_adv_data.clone(), **defense_params)
                    pgd_defended_output = model(pgd_defended_data)
                    pgd_defended_pred = torch.argmax(pgd_defended_output, dim=1)
                    correct_pgd_with_def = (pgd_defended_pred == target).sum().item()
                    results[epsilon]['pgd_with_defense'] += correct_pgd_with_def
                else:
                    results[epsilon]['pgd_with_defense'] += correct_pgd_no_def
        
        total += target.size(0)
    
    # Convert to percentages
    clean_accuracy = 100. * clean_correct / total
    
    for epsilon in epsilons:
        for key in results[epsilon]:
            results[epsilon][key] = 100. * results[epsilon][key] / total
    
    return clean_accuracy, results

print("📊 COMPREHENSIVE DEFENSE EVALUATION")
print("=" * 60)

# Test different defense strategies
defense_strategies = [
    {
        'name': 'No Defense',
        'function': None,
        'params': {}
    },
    {
        'name': 'Gaussian Noise (σ=0.1)',
        'function': gaussian_noise_defense,
        'params': {'noise_std': 0.1}
    },
    {
        'name': 'Gaussian Noise (σ=0.2)',
        'function': gaussian_noise_defense,
        'params': {'noise_std': 0.2}
    }
]

# Evaluate each defense strategy
defense_results = {}

print("\n🔍 Testing Standard Model with Different Defenses:")
print("-" * 50)

for strategy in defense_strategies:
    print(f"\nTesting: {strategy['name']}")
    
    clean_acc, adv_results = evaluate_defense_robustness(
        model, test_loader_single, 
        strategy['function'], 
        strategy['params'], 
        num_samples=100
    )
    
    defense_results[strategy['name']] = {
        'clean_accuracy': clean_acc,
        'adversarial_results': adv_results
    }
    
    print(f"  Clean Accuracy: {clean_acc:.1f}%")
    for eps in [0.1, 0.2, 0.3]:
        if strategy['function'] is None:
            print(f"  FGSM Accuracy (ε={eps}): {adv_results[eps]['fgsm_no_defense']:.1f}%")
            print(f"  PGD Accuracy (ε={eps}): {adv_results[eps]['pgd_no_defense']:.1f}%")
        else:
            print(f"  FGSM Accuracy (ε={eps}): {adv_results[eps]['fgsm_no_defense']:.1f}% → {adv_results[eps]['fgsm_with_defense']:.1f}%")
            print(f"  PGD Accuracy (ε={eps}): {adv_results[eps]['pgd_no_defense']:.1f}% → {adv_results[eps]['pgd_with_defense']:.1f}%")

print("\n🛡️ Testing Robust Model (Adversarial Training):")
print("-" * 50)

# Test the robust model
robust_clean_acc, robust_adv_results = evaluate_defense_robustness(
    robust_model, test_loader_single, 
    None, {}, num_samples=100
)

print(f"Robust Model Performance:")
print(f"  Clean Accuracy: {robust_clean_acc:.1f}%")
for eps in [0.1, 0.2, 0.3]:
    print(f"  FGSM Accuracy (ε={eps}): {robust_adv_results[eps]['fgsm_no_defense']:.1f}%")
    print(f"  PGD Accuracy (ε={eps}): {robust_adv_results[eps]['pgd_no_defense']:.1f}%")

defense_results['Adversarial Training'] = {
    'clean_accuracy': robust_clean_acc,
    'adversarial_results': robust_adv_results
}

print("\n✅ Defense evaluation complete!")


## Summary

**Congratulations!** You have successfully implemented and evaluated multiple adversarial defense strategies.
